## An example of model training and evaluation
Author: Yarden Cohen

Date: July 2020

The script will load trainig data for a Bengalese finch (gy6or6, date: 032212), train a TweetyNet model, and predict a test set.

Data source: https://figshare.com/articles/Bengalese_Finch_song_repository/4805749

Before running:
1. Download the data from 032212.
2. Change the folder definitions in the configuration files to the path for the data and the example csv file


In [1]:
# imports
from pathlib import Path
import pandas as pd
import numpy as np
import json
import torch.utils.data
from argparse import ArgumentParser
from collections import defaultdict
import joblib
import torch
from tqdm import tqdm

# imports from vak
from vak import csv,labels,models,summary_writer,transforms,io,config
from vak.datasets.window_dataset import WindowDataset
from vak.datasets.vocal_dataset import VocalDataset
from vak.device import get_default as get_default_device
from vak.logging import log_or_print
from vak.config import parse
from vak.io import dataframe
import vak.device
import vak.files
import vak.labels as labelfuncs

In [2]:
# setting up parameters for training
path_of_configuration_file = "C:\\Users\\Yarden Cohen\\repos\\tweetynet\\doc\\notebooks\\BF_Example_Train_Predict\\gy6or6_032212_example_train.toml"

toml_path = Path(path_of_configuration_file)
cfg = parse.from_toml(toml_path)

model_config_map = {'TweetyNet': {'loss': {}, 'metrics': {}, 'network': {}, 'optimizer': {'lr': 0.001}}}
train_csv_path = Path(cfg.train.csv_path)
labelset =  cfg.prep.labelset
window_size = cfg.dataloader.window_size
batch_size = cfg.train.batch_size
num_epochs = cfg.train.num_epochs
num_workers = cfg.train.num_workers
results_path = Path(cfg.train.root_results_dir)
spect_key = cfg.spect_params.spect_key
timebins_key = cfg.spect_params.timebins_key
normalize_spectrograms = cfg.train.normalize_spectrograms
shuffle = cfg.train.shuffle
val_step = cfg.train.val_step
ckpt_step = cfg.train.ckpt_step
patience = cfg.train.patience
device = cfg.train.device
logger = None

In [27]:
# make spectrograms and update path in example csv file
spect_path = cfg.prep.data_dir.joinpath('spect')
if not spect_path.is_dir():
    spect_path.mkdir()
dataset_df=dataframe.from_files(labelset=cfg.prep.labelset,
                          data_dir=cfg.prep.data_dir,
                          annot_format=cfg.prep.annot_format,
                          output_dir=spect_path,
                          annot_file=None,
                          audio_format='cbin',
                          spect_params=cfg.spect_params,
                          logger=None)
example_csv_path = cfg.train.csv_path
example_csv_df = pd.read_csv(example_csv_path)
for cnt in range(len(example_csv_df)):
    if example_csv_df['audio_path'][cnt] == dataset_df['audio_path'][cnt]:
        example_csv_df['spect_path'][cnt] = dataset_df['spect_path'][cnt]
    else:
        print('Audio file name mismatch in entry: ' + str(cnt))
example_csv_df.to_csv(example_csv_path,index=False)

making array files containing spectrograms from audio files in: D:\BengaleseFinches\gy6or6\032212
found labels in D:\BengaleseFinches\gy6or6\032212\gy6or6_baseline_220312_0844.22.cbin.not.mat for D:\BengaleseFinches\gy6or6\032212\gy6or6_baseline_220312_0844.22.cbin not in labels_mapping, skipping audio file: D:\BengaleseFinches\gy6or6\032212\gy6or6_baseline_220312_0844.22.cbin
found labels in D:\BengaleseFinches\gy6or6\032212\gy6or6_baseline_220312_0845.28.cbin.not.mat for D:\BengaleseFinches\gy6or6\032212\gy6or6_baseline_220312_0845.28.cbin not in labels_mapping, skipping audio file: D:\BengaleseFinches\gy6or6\032212\gy6or6_baseline_220312_0845.28.cbin
found labels in D:\BengaleseFinches\gy6or6\032212\gy6or6_baseline_220312_0852.55.cbin.not.mat for D:\BengaleseFinches\gy6or6\032212\gy6or6_baseline_220312_0852.55.cbin not in labels_mapping, skipping audio file: D:\BengaleseFinches\gy6or6\032212\gy6or6_baseline_220312_0852.55.cbin
found labels in D:\BengaleseFinches\gy6or6\032212\gy6or6

In [45]:
# prepare training dataset
dataset_df = pd.read_csv(train_csv_path)
results_path = Path(results_path).expanduser().resolve()
timebin_dur = dataframe.validate_and_get_timebin_dur(dataset_df)
train_dur = dataframe.split_dur(dataset_df, 'train')
labelmap = labels.to_map(labelset, map_unlabeled=True)
spect_standardizer = None
transform, target_transform = transforms.get_defaults('train',spect_standardizer)
train_dataset = WindowDataset.from_csv(csv_path=train_csv_path,
                                           x_inds=None,
                                           spect_id_vector=None,
                                           spect_inds_vector=None,
                                           split='train',
                                           labelmap=labelmap,
                                           window_size=window_size,
                                           spect_key=spect_key,
                                           timebins_key=timebins_key,
                                           transform=transform,
                                           target_transform=target_transform
                                           )

train_data = torch.utils.data.DataLoader(dataset=train_dataset,
                                             shuffle=shuffle,
                                             batch_size=batch_size,
                                             num_workers=num_workers)

In [47]:
# prepare validation dataset 
if val_step:
    item_transform = transforms.get_defaults('eval',
                                             spect_standardizer,
                                             window_size=window_size,
                                             return_padding_mask=True,
                                             )
    val_dataset = VocalDataset.from_csv(csv_path=train_csv_path,
                                        split='val',
                                        labelmap=labelmap,
                                        spect_key=spect_key,
                                        timebins_key=timebins_key,
                                        item_transform=item_transform,
                                        )
    val_data = torch.utils.data.DataLoader(dataset=val_dataset,
                                           shuffle=False,
                                           # batch size 1 because each spectrogram reshaped into a batch of windows
                                           batch_size=1,
                                           num_workers=num_workers)
    val_dur = dataframe.split_dur(dataset_df, 'val')    
else:
    val_data = None

In [50]:
# initiate the TweetyNet model and prepare for training 
if device is None:
    device = get_default_device()

models_map = models.from_model_config_map(
        model_config_map,
        num_classes=len(labelmap),
        input_shape=train_dataset.shape,
        logger=logger,
    )

model_name = 'TweetyNet'
model = models_map['TweetyNet']
results_model_root = results_path.joinpath(model_name)
if not results_model_root.is_dir():
    results_model_root.mkdir()
ckpt_root = results_model_root.joinpath('checkpoints')
if not ckpt_root.is_dir():
    ckpt_root.mkdir()
writer = summary_writer.get_summary_writer(log_dir=results_model_root,                                                 filename_suffix=model_name)
model.summary_writer = writer

In [51]:
# This is how the model looks like
model.network

TweetyNet(
  (cnn): Sequential(
    (0): Conv2dTF(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=same)
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=(8, 1), stride=(8, 1), padding=0, dilation=1, ceil_mode=False)
    (3): Conv2dTF(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=same)
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=(8, 1), stride=(8, 1), padding=0, dilation=1, ceil_mode=False)
  )
  (rnn): LSTM(128, 128, bidirectional=True)
  (fc): Linear(in_features=256, out_features=12, bias=True)
)

In [7]:
# train the model
model.fit(train_data=train_data,
          num_epochs=num_epochs,
          ckpt_root=ckpt_root,
          val_data=val_data,
          val_step=val_step,
          ckpt_step=ckpt_step,
          patience=patience,
          device=device)

  0%|                                                                                        | 0/11862 [00:00<?, ?it/s]

epoch 1 / 5


Epoch 1, batch 49. Loss: 0.6468. Global step: 49:   0%|                           | 46/11862 [00:45<2:58:07,  1.11it/s]
  0%|                                                                                            | 0/8 [00:00<?, ?it/s]

Step 50 is a validation step; computing metrics on validation set


Epoch 1, batch 49. Loss: 0.6468. Global step: 49:   0%|                           | 46/11862 [00:59<2:58:07,  1.11it/s]
batch 7 / 8: 100%|███████████████████████████████████████████████████████████████████████| 8/8 [00:44<00:00,  5.55s/it]
Epoch 1, batch 53. Loss: 0.9127. Global step: 53:   0%|                           | 53/11862 [01:30<9:09:27,  2.79s/it]

...

avg_acc: 0.9764, avg_levenshtein: 13.0000, avg_segment_error_rate: 0.1966, avg_loss: 0.0759
Accuracy has not improved in 4 validation steps. Not saving max-val-acc checkpoint for this validation step.


Epoch 1, batch 1549. Loss: 0.0461. Global step: 1549:  13%|██▉                    | 1546/11862 [24:15<16:45, 10.26it/s]
  0%|                                                                                            | 0/8 [00:00<?, ?it/s]

Step 1550 is a validation step; computing metrics on validation set


Epoch 1, batch 1549. Loss: 0.0461. Global step: 1549:  13%|██▉                    | 1546/11862 [24:30<16:45, 10.26it/s]
batch 7 / 8: 100%|███████████████████████████████████████████████████████████████████████| 8/8 [00:44<00:00,  5.51s/it]
Epoch 1, batch 1549. Loss: 0.0461. Global step: 1549:  13%|██▋                  | 1546/11862 [24:59<2:46:47,  1.03it/s]


avg_acc: 0.9742, avg_levenshtein: 17.0000, avg_segment_error_rate: 0.2630, avg_loss: 0.0805
Stopping training early, accuracy has not improved in 4 validation steps.
Saving checkpoint at:
C:\Users\Yarden Cohen\repos\tweetynet\doc\notebooks\BF_Example_Train_Predict\TweetyNet\checkpoints\checkpoint.pt 


Now that we have a trained model we can use it to predict segments and labels in a test set

In [94]:
# setting up parameters for prediction
path_of_configuration_file = "C:\\Users\\Yarden Cohen\\repos\\tweetynet\\doc\\notebooks\\BF_Example_Train_Predict\\gy6or6_032212_example_eval.toml"

toml_path = Path(path_of_configuration_file)
cfg = parse.from_toml(toml_path)
min_segment_dur = 0.01
device='cuda'
spect_key='s'
timebins_key='t'
spect_standardizer = joblib.load(cfg.eval.spect_scaler_path)
with cfg.eval.labelmap_path.open('r') as f:
        labelmap = json.load(f)

In [95]:
# prepare evaluation data
item_transform = transforms.get_defaults('eval',
                                             spect_standardizer=None,
                                             window_size=cfg.dataloader.window_size,
                                             return_padding_mask=True,
                                             )

eval_dataset = VocalDataset.from_csv(csv_path=cfg.eval.csv_path,
                                     split='test',
                                     labelmap=labelmap,
                                     spect_key=spect_key,
                                     timebins_key=timebins_key,
                                     item_transform=item_transform,
                                     )

eval_data = torch.utils.data.DataLoader(dataset=eval_dataset,
                                        shuffle=False,
                                        # batch size 1 because each spectrogram reshaped into a batch of windows
                                        batch_size=1,
                                        num_workers=cfg.eval.num_workers)


In [96]:
# Create model
input_shape = eval_dataset.shape
# if dataset returns spectrogram reshaped into windows,
# throw out the window dimension; just want to tell network (channels, height, width) shape
if len(input_shape) == 4:
    input_shape = input_shape[1:]

models_map = models.from_model_config_map(
    model_config_map,
    num_classes=len(labelmap),
    input_shape=input_shape
)
model_name = 'TweetyNet'
model = models_map['TweetyNet']
model.load(cfg.eval.checkpoint_path)
metrics = model.metrics  # metric name -> callable map we use below in loop
if device is None:
    device = vak.device.get_default_device()
pred_dict = model.predict(pred_data=eval_data,
                          device=device)




  0%|                                                                                           | 0/17 [00:00<?, ?it/s]

Loading checkpoint from:
C:\Users\Yarden Cohen\repos\tweetynet\doc\notebooks\BF_Example_Train_Predict\TweetyNet\checkpoints\max-val-acc-checkpoint.pt 





batch 0 / 17:   0%|                                                                             | 0/17 [00:42<?, ?it/s]


batch 0 / 17:   6%|████                                                                 | 1/17 [00:42<11:19, 42.44s/it]


batch 1 / 17:   6%|████                                                                 | 1/17 [00:42<11:19, 42.44s/it]


batch 2 / 17:   6%|████                                                                 | 1/17 [00:42<11:19, 42.44s/it]


batch 3 / 17:   6%|████                                                                 | 1/17 [00:42<11:19, 42.44s/it]


batch 4 / 17:   6%|████                                                                 | 1/17 [00:42<11:19, 42.44s/it]


batch 5 / 17:   6%|████                                                                 | 1/17 [00:42<11:19, 42.44s/it]


batch 6 / 17:   6%|████                                                                 | 1/17 [00:42<11:19, 42.44s/it]


batch 7 / 17:   6%|██

In [97]:
def compute_metrics(metrics, y_true, y_pred, y_true_labels, y_pred_labels):
    """helper function to compute metrics

    Parameters
    ----------
    metrics : dict
        where keys are metric names and values are callables that compute the metric
        given ground truth and prediction
    y_true : torch.Tensor
        vector of labeled time bins
    y_pred : torch.Tensor
        vector of labeled time bins
    y_true_labels : str
        sequence of segment labels
    y_pred_labels : str
        sequence of segment labels

    Returns
    -------
    metric_vals : defaultdict
    """
    metric_vals = {}

    for metric_name, metric_callable in metrics.items():
        if metric_name == 'acc':
            metric_vals[metric_name] = metric_callable(y_pred, y_true)
        elif metric_name == 'levenshtein':
            metric_vals[metric_name] = metric_callable(y_pred_labels, y_true_labels)
        elif metric_name == 'segment_error_rate':
            metric_vals[metric_name] = metric_callable(y_pred_labels, y_true_labels)

    return metric_vals

In [98]:
records = defaultdict(list)  # will be used with pandas.DataFrame.from_records to make output csv
to_long_tensor = transforms.ToLongTensor()
progress_bar = tqdm(eval_data)
for ind, batch in enumerate(progress_bar):
    y_true, padding_mask, spect_path = batch['annot'], batch['padding_mask'], batch['spect_path']
    # need to convert spect_path to tuple for match in call to index() below
    spect_path = tuple(spect_path)
    records['spect_path'].append(spect_path[0])  # remove str from tuple
    y_true = y_true.to(device)
    y_true_np = np.squeeze(y_true.cpu().numpy())
    y_true_labels, _, _ = labelfuncs.lbl_tb2segments(y_true_np,
                                                     labelmap=labelmap,
                                                     timebin_dur=timebin_dur)
    y_true_labels = ''.join(y_true_labels.tolist())

    y_pred_ind = spect_path[0] #pred_dict['y'].index(spect_path)
    y_pred = pred_dict[y_pred_ind] #pred_dict['y_pred'][y_pred_ind]
    y_pred = torch.argmax(y_pred, dim=1)  # assumes class dimension is 1
    y_pred = torch.flatten(y_pred)
    y_pred = y_pred.unsqueeze(0)[padding_mask]
    y_pred_np = np.squeeze(y_pred.cpu().numpy())
    y_pred_labels, _, _ = labelfuncs.lbl_tb2segments(y_pred_np,
                                                     labelmap=labelmap,
                                                     timebin_dur=timebin_dur,
                                                     min_segment_dur=None,
                                                     majority_vote=False)
    y_pred_labels = ''.join(y_pred_labels.tolist())

    metric_vals_batch = compute_metrics(metrics, y_true, y_pred, y_true_labels, y_pred_labels)
    for metric_name, metric_val in metric_vals_batch.items():
        records[metric_name].append(metric_val)

    # --- apply majority vote and min segment dur transforms separately
    # need segment_inds_list for both transforms
    segment_inds_list = labelfuncs.lbl_tb_segment_inds_list(y_pred_np,
                                                            unlabeled_label=labelmap['unlabeled'])

    # ---- majority vote transform
    y_pred_np_mv = labelfuncs.majority_vote_transform(y_pred_np, segment_inds_list)
    y_pred_mv = to_long_tensor(y_pred_np_mv).to(device)
    y_pred_mv_labels, _, _ = labelfuncs.lbl_tb2segments(y_pred_np_mv,
                                                        labelmap=labelmap,
                                                        timebin_dur=timebin_dur,
                                                        min_segment_dur=None,
                                                        majority_vote=False)
    y_pred_mv_labels = ''.join(y_pred_mv_labels.tolist())
    metric_vals_batch_mv = compute_metrics(metrics, y_true, y_pred_mv,
                                           y_true_labels, y_pred_mv_labels)
    for metric_name, metric_val in metric_vals_batch_mv.items():
        records[f'{metric_name}_majority_vote'].append(metric_val)

    # ---- min segment dur transform
    y_pred_np_mindur, _ = labelfuncs.remove_short_segments(y_pred_np,
                                                           segment_inds_list,
                                                           timebin_dur=timebin_dur,
                                                           min_segment_dur=min_segment_dur,
                                                           unlabeled_label=labelmap['unlabeled'])
    y_pred_mindur = to_long_tensor(y_pred_np_mindur).to(device)
    y_pred_mindur_labels, _, _ = labelfuncs.lbl_tb2segments(y_pred_np_mindur,
                                                            labelmap=labelmap,
                                                            timebin_dur=timebin_dur,
                                                            min_segment_dur=None,
                                                            majority_vote=False)
    y_pred_mindur_labels = ''.join(y_pred_mindur_labels.tolist())
    metric_vals_batch_mindur = compute_metrics(metrics, y_true, y_pred_mindur,
                                               y_true_labels, y_pred_mindur_labels)
    for metric_name, metric_val in metric_vals_batch_mindur.items():
        records[f'{metric_name}_min_segment_dur'].append(metric_val)

    # ---- and finally both transforms, in same order we apply for prediction
    y_pred_np_mindur_mv, segment_inds_list = labelfuncs.remove_short_segments(y_pred_np,
                                                                              segment_inds_list,
                                                                              timebin_dur=timebin_dur,
                                                                              min_segment_dur=min_segment_dur,
                                                                              unlabeled_label=labelmap[
                                                                                  'unlabeled'])
    y_pred_np_mindur_mv = labelfuncs.majority_vote_transform(y_pred_np_mindur_mv,
                                                             segment_inds_list)
    y_pred_mindur_mv = to_long_tensor(y_pred_np_mindur_mv).to(device)
    y_pred_mindur_mv_labels, _, _ = labelfuncs.lbl_tb2segments(y_pred_np_mindur_mv,
                                                               labelmap=labelmap,
                                                               timebin_dur=timebin_dur,
                                                               min_segment_dur=None,
                                                               majority_vote=False)
    y_pred_mindur_mv_labels = ''.join(y_pred_mindur_mv_labels.tolist())
    metric_vals_batch_mindur_mv = compute_metrics(metrics, y_true, y_pred_mindur_mv,
                                                  y_true_labels, y_pred_mindur_mv_labels)
    for metric_name, metric_val in metric_vals_batch_mindur_mv.items():
        records[f'{metric_name}_min_dur_maj_vote'].append(metric_val)
df = pd.DataFrame.from_records(records)




  0%|                                                                                           | 0/17 [00:00<?, ?it/s]


  6%|████▉                                                                              | 1/17 [00:44<11:46, 44.15s/it]


 41%|██████████████████████████████████▏                                                | 7/17 [00:44<05:09, 30.91s/it]


 71%|█████████████████████████████████████████████████████████▉                        | 12/17 [00:44<01:48, 21.64s/it]


100%|██████████████████████████████████████████████████████████████████████████████████| 17/17 [00:45<00:00,  2.67s/it]


In [99]:
df

,acc,acc_majority_vote,acc_min_dur_maj_vote,acc_min_segment_dur,levenshtein,levenshtein_majority_vote,levenshtein_min_dur_maj_vote,levenshtein_min_segment_dur,segment_error_rate,segment_error_rate_majority_vote,segment_error_rate_min_dur_maj_vote,segment_error_rate_min_segment_dur,spect_path
0,0.976961,0.980618,0.981167,0.981167,17,2,1,1,0.250000,0.029412,0.014706,0.014706,D:\BengaleseFinches\gy6or6\vak\spectrograms_ge...
1,0.980854,0.983544,0.983544,0.983544,8,1,1,1,0.101266,0.012658,0.012658,0.012658,D:\BengaleseFinches\gy6or6\vak\spectrograms_ge...
2,0.983028,0.984708,0.985213,0.985213,14,3,0,0,0.208955,0.044776,0.000000,0.000000,D:\BengaleseFinches\gy6or6\vak\spectrograms_ge...
3,0.978516,0.980505,0.980505,0.980505,11,1,1,1,0.174603,0.015873,0.015873,0.015873,D:\BengaleseFinches\gy6or6\vak\spectrograms_ge...
4,0.975496,0.978614,0.979283,0.979283,11,1,0,0,0.200000,0.018182,0.000000,0.000000,D:\BengaleseFinches\gy6or6\vak\spectrograms_ge...
5,0.968452,0.971040,0.971526,0.971526,25,6,4,4,0.308642,0.074074,0.049383,0.049383,D:\BengaleseFinches\gy6or6\vak\spectrograms_ge...
6,0.980421,0.981889,0.981889,0.981889,10,3,3,3,0.129870,0.038961,0.038961,0.038961,D:\BengaleseFinches\gy6or6\vak\spectrograms_ge...
7,0.978057,0.982140,0.982650,0.982650,16,2,0,0,0.219178,0.027397,0.000000,0.000000,D:\BengaleseFinches\gy6or6\vak\spectrograms_ge...
8,0.957321,0.961261,0.964051,0.964051,34,18,12,12,0.523077,0.276923,0.184615,0.184615,D:\BengaleseFinches\gy6or6\vak\spectrograms_ge...
9,0.982392,0.983790,0.984908,0.984908,11,4,1,1,0.122222,0.044444,0.011111,0.011111,D:\BengaleseFinches\gy6or6\vak\spectrograms_ge...
